# Introduction to Natural Language Processing with fastText
In this notebook we will discuss what is Natural Language Processing (NLP) and how to easily implement several projects using the library [fastText](https://github.com/facebookresearch/fastText).

In [2]:
#Load all libraries
import os,sys  
import pandas as pd
import numpy as np
import fasttext

print(sys.version)

3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


## Text classification
The first task will be to perform text classification dataset DBPedia. 

In [94]:
#Load data
train_file = 'dbpedia_train.csv'
df = pd.read_csv(train_file, header=None, names=['class','name','description'])
class_dict={
1:'Company',
2:'EducationalInstitution',
3:'Artist',
4:'Athlete',
5:'OfficeHolder',
6:'MeanOfTransportation',
7:'Building',
8:'NaturalPlace',
9:'Village',
10:'Animal',
11:'Plant',
12:'Album',
13:'Film',
14:'WrittenWork'
}
df['class_name'] = df['class'].map(class_dict)
df.head()

,class,name,description,class_name
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...,Company
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...,Company
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...,Company
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...,Company
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...,Company


In [24]:
#df.describe().transpose()
desc = df.groupby('class')
desc.describe().transpose()

class           1                                                             \
             count unique                                                top   
class_name   40000      1                                            Company   
description  40000  39996   The Spanish Royal Society of Chemistry (RSEQ)...   
name         40000  40000                                         Blue Arrow   

class                  2          \
              freq  count unique   
class_name   40000  40000      1   
description      2  40000  39992   
name             1  40000  40000   

class                                                                     3   \
                                                           top   freq  count   
class_name                              EducationalInstitution  40000  40000   
description   R.B. Govt. High School (Bengali: রামদেও বাজলা...      2  40000   
name                                      Gavar Special School      1  40000   

class               ...                                                   12  \
            unique  ...                                                  top   
class_name       1  ...                                                Album   
description  40000  ...     Before Smile Empty Soul became Smile Empty So...   
name         40000  ...                                  2nd (The Rasmus EP)   

class                  13         \
              freq  count unique   
class_name   40000  40000      1   
description      2  40000  40000   
name             1  40000  40000   

class                                                                     14  \
                                                           top   freq  count   
class_name                                                Film  40000  40000   
description   Vagabond (French: Sans toit ni loi without ro...      1  40000   
name                                  Eye of the Needle (film)      1  40000   

class                                                                         
            unique                                                top   freq  
class_name       1                                        WrittenWork  40000  
description  39984   Tom Clancy's Net Force Explorers or Net Force...     15  
name         40000                                   Er ist wieder da      1  

[3 rows x 56 columns]

The next step is to treat the data. As of today, the python wrapper of fastText doesn't allow dataframes or iterators as inputs to their functions (however, they are [working on it](https://github.com/salestock/fastText.py/issues/78). We have to create an intermediate file. This intermediate file doesn't have commas, non-ascii characters and everything is lowercase. 

In [95]:
#Load test set
test_file = 'dbpedia_test.csv'
df_test = pd.read_csv(train_file, header=None, names=['class','name','description'])

# Transform train file
df[['name','description']] = df[['name','description']].replace({',':' '})
df[['name','description']] = df[['name','description']].apply(lambda x: x.str.lower())
df['class'] = '__label__' + df['class'].astype(str)
df.drop['class_name']

# Transform test file


# Write files to disk
train_file_clean = 'dbpedia.train'
test_file_clean = 'dbpedia.test'

In [33]:
%%time
# Train a classifier
output_file = 'dp_model'
classifier = fasttext.supervised('data/dbpedia.train', output_file, label_prefix='__label__')

CPU times: user 1min 28s, sys: 1.1 s, total: 1min 29s
Wall time: 20.3 s


In [34]:
%%time
# Evaluate classifier
test_file = 'data/dbpedia.test'
result = classifier.test(test_file)
print('P@1:', result.precision)
print('R@1:', result.recall)
print ('Number of examples:', result.nexamples)

P@1: 0.9834571428571428
R@1: 0.9834571428571428
Number of examples: 70000
CPU times: user 580 ms, sys: 12 ms, total: 592 ms
Wall time: 591 ms


The next step is to check how the model works with real sentences.

In [78]:
sentence1 = ['Picasso was a famous painter born in Malaga, Spain. He revolutionized the art in the 20th century.']
labels1 = classifier.predict(sentence1)
class1 = int(labels1[0][0])
print("Sentence: ", sentence1[0])
print("Label: %d; label name: %s" %(class1, class_dict[class1]))

sentence2 = ['One of my favourite tennis players in the world is Rafa Nadal.']
labels2 = classifier.predict_proba(sentence2)
class2, prob2 = labels2[0][0] # it returns class2 as string
print("Sentence: ", sentence2[0])
print("Label: %s; label name: %s; certainty: %f" %(class2, class_dict[int(class2)], prob2))

sentence3 = ['Say what one more time, I dare you, I double-dare you motherfucker!']
number_responses = 3
labels3 = classifier.predict_proba(sentence3, k=number_responses)
print("Sentence: ", sentence3[0])
for l in range(number_responses):
    class3, prob3 = labels3[0][l]
    print("Label: %s; label name: %s; certainty: %f" %(class3, class_dict[int(class3)], prob3))


Sentence:  Picasso was a famous painter born in Malaga, Spain. He revolutionized the art in the 20th century.
Label: 3; label name: Artist
Sentence:  One of my favourite tennis players in the world is Rafa Nadal.
Label: 4; label name: Athlete; certainty: 0.449219
Sentence:  Say what one more time, I dare you, I double-dare you motherfucker!
Label: 3; label name: Artist; certainty: 0.691406
Label: 14; label name: WrittenWork; certainty: 0.181641
Label: 12; label name: Album; certainty: 0.068359


The model predicts sentence 1 as `Artist`, which is correct. Sentence 2 is also predicted correctly. This time we used the function `predict_proba` that retruns the certainty of the prediction as a probability. Finally, sentence 3 was not correctly classified. The correct label would be `Film`, since the sentence is from famous scene of a very good film. If by any chance, you don't know [what I'm talking about](https://www.youtube.com/watch?v=xwT60UbOZnI), well, please put your priorities in order. Stop reading this notebook, go to see Pulp Fiction, and then come back to keep learning NLP :-)

In [96]:
df

,class,name,description,class_name
0,__label__1,e. d. abbott ltd,abbott of farnham e d abbott limited was a br...,Company
1,__label__1,schwan-stabilo,schwan-stabilo is a german maker of pens for ...,Company
2,__label__1,q-workshop,q-workshop is a polish company located in poz...,Company
3,__label__1,marvell software solutions israel,marvell software solutions israel known as ra...,Company
4,__label__1,bergan mercy medical center,bergan mercy medical center is a hospital loc...,Company
5,__label__1,the unsigned guide,the unsigned guide is an online contacts dire...,Company
6,__label__1,rest of the world,within sports and games played at the interna...,Company
7,__label__1,globoforce,globoforce is a multinational company co-head...,Company
8,__label__1,rompetrol,the rompetrol group n.v. is a romanian oil co...,Company
9,__label__1,wave accounting,wave is the brand name for a suite of online ...,Company
